<a href="https://colab.research.google.com/github/Megaexoplanet/project-3/blob/master/PThree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Basic libaries
import numpy as np
from datetime import datetime as dt, timedelta as td
# Data manipulation
import pandas as pd
import csv
# Modelling
import statsmodels.api as sm
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
import scipy as sp
import tensorflow as tf
# Visualizations
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
matplotlib.use('SVG')

In [ ]:
# Changing Frequency
freq=td(hours=1)
# Choose missing_data_handling to be True to fill missing data points with a concave function and False to remove missing data points.
missing_data_handling=False

In [ ]:
# reading in stock index date
prices=pd.read_csv("prices_60m.csv",parse_dates=['Dates'])
prices.rename(columns={'Dates':'dates'},inplace=True)
prices=prices.set_index('dates')
prices=prices.dropna(how='any')
prices

if missing_data_handling:
    prices=prices.resample(freq,base=15.5).asfreq()
    
# calculating logreturns and Z Scores
def logreturn(x):
    return np.log(x/x.shift(1))

def zscore(x):
    return (x-x.mean())/x.std()

prices['DJIA LogRet']=logreturn(prices['DJIA CLOSE'])
prices['SP500 LogRet']=logreturn(prices['SP500 CLOSE'])
prices['DJIA Z Scores']=zscore(prices['DJIA CLOSE'])
prices['SP500 Z Scores']=zscore(prices['SP500 CLOSE'])
prices['DJIA Log']=np.log(prices['DJIA CLOSE'])
prices['SP500 Log']=np.log(prices['SP500 CLOSE'])
prices=prices.dropna(how='any')
prices

,DJIA CLOSE,SP500 CLOSE,DJIA LogRet,SP500 LogRet,DJIA Z Scores,SP500 Z Scores,DJIA Log,SP500 Log
dates,,,,,,,,
2019-06-21 17:30:00,26791.51,2952.75,-0.001901,-0.001739,0.241810,-0.054862,10.195840,7.990492
2019-06-21 18:30:00,26796.05,2953.07,0.000169,0.000108,0.250797,-0.048422,10.196010,7.990601
2019-06-21 19:30:00,26790.67,2953.51,-0.000201,0.000149,0.240148,-0.039568,10.195809,7.990750
2019-06-21 20:30:00,26793.13,2955.41,0.000092,0.000643,0.245017,-0.001331,10.195901,7.991393
2019-06-21 21:30:00,26719.13,2950.46,-0.002766,-0.001676,0.098545,-0.100948,10.193135,7.989716
...,...,...,...,...,...,...,...,...
2019-09-27 17:30:00,26890.71,2970.09,-0.004136,-0.004612,0.438163,0.294099,10.199536,7.996348
2019-09-27 18:30:00,26825.86,2961.04,-0.002415,-0.003052,0.309801,0.111971,10.197122,7.993296
2019-09-27 19:30:00,26776.07,2954.41,-0.001858,-0.002242,0.211249,-0.021455,10.195264,7.991054


In [ ]:
# Reading in csv file containing tweets and converting it to a pandas dataframe
tweets=pd.read_csv("raw_tweets.csv",parse_dates=['created_at'])
tweets = tweets.drop(columns="id")

# Converting time from UTC to CEST +2 hours
cest_date=[]
for date in tweets['created_at']:
    cest_date.append(date+td(hours=2))
tweets['created_at']=cest_date

# Changing index to Datetimeindex
tweets.rename(columns={'created_at':'date'},inplace=True)
tweets=tweets.set_index('date')
#tweets.index = pd.DatetimeIndex(tweets.index)

tweets

,tweet_text,bullish
date,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession ri...,NaN
2019-06-24 19:58:00,@nyika_yedu The multi currency regime dealt wi...,NaN
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying ...,NaN
2019-06-24 19:58:00,"The caution continues. ""Small business owners ...",NaN
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - htt...,NaN
...,...,...
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession deni...,NaN
2019-09-03 20:47:00,NaN,True
2019-09-03 20:47:00,@pdtugs @CPC_HQ Actually Canada economy is goo...,NaN


In [ ]:
len(tweets)

1048575

In [ ]:
tweets['Time']=tweets.index.hour

In [ ]:
df=tweets.groupby(tweets.index.date).count()
df['Total']=df['tweet_text']+df['bullish']
df['Total'].mean()

14563.541666666666

In [ ]:
#tweets['Time'].value_counts().sort_index().plot.bar()
fig, ax = plt.subplots()
counts, bins, patches = ax.hist(tweets.Time, facecolor='gray', edgecolor='black', bins=range(0,25))
ax.set_xticks(bins)
ax.set_title('Tweetek óránkénti eloszlása', fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=7)
plt.savefig("hourly_tweet_distribution.svg", format="svg")
print(counts)
print(bins)

[43588. 38517. 37106. 34865. 33845. 31385. 28502. 25502. 28511. 31300.
 29118. 29834. 36889. 41597. 55131. 61533. 63697. 62545. 61488. 56832.
 55641. 54242. 57459. 49448.]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]


In [ ]:
if not missing_data_handling:
    # Removing tweets that were created on weekends and bank holidays
    #getting trading dates from DJIA intraday prices
    t_dates=prices.index.map(pd.Timestamp.date).unique()
    str_t_dates=[str(x) for x in t_dates]
    tweets=tweets[tweets.index.floor('D').isin(str_t_dates)]
    # Removing tweets that are not in trading analysis interval
    start=str(td(hours=15,minutes=30)-freq)
    end=str(td(hours=21,minutes=29))
    tweets = tweets.between_time(start, end, include_start=True, include_end=True)

In [ ]:
# Separating sentiment tweets from bullish/bearish index tweets
text_tweets=tweets.loc[:, ['tweet_text']]
market_tweets=tweets.loc[:, ['bullish']]

In [ ]:
# Filtering out NaNs
fmarket_tweets=market_tweets[market_tweets['bullish'].notnull()]
ftext_tweets=text_tweets[text_tweets['tweet_text'].notnull()]
ftext_tweets =ftext_tweets.drop_duplicates(subset="tweet_text",keep=False)
print('Total number of observations including both text and bullish/bearish tweets:',ftext_tweets.tweet_text.count()+fmarket_tweets.bullish.count())
ftext_tweets

Total number of observations including both text and bullish/bearish tweets: 327195


,tweet_text
date,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession ri...
2019-06-24 19:58:00,@nyika_yedu The multi currency regime dealt wi...
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying ...
2019-06-24 19:58:00,"The caution continues. ""Small business owners ..."
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - htt...
...,...
2019-09-03 20:47:00,"@PrimeXbt In general, the SP500 is clearly ove..."
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession deni...
2019-09-03 20:47:00,@pdtugs @CPC_HQ Actually Canada economy is goo...


In [ ]:
text = " ".join(tweet for tweet in ftext_tweets.tweet_text)
print ('There are %s words in the combination of all review.' % (len(text)))

There are 49648007 words in the combination of all review.


In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords_raw=["one","co","amp","https","bullish", "bearish", "stock market", "I think economy", "I feel economy", "I am feeling economy", "It feels economy","inflation", "unemployment rate", "recession","SP500","S&P500","DJIA","Dow jones"]
stopwords_splited=[]
for phrase in stopwords_raw:
    split=phrase.split()
    stopwords_splited=list(set(stopwords_splited+split))
stopwords.update(stopwords_splited)

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white",width=800, height=400).generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
fig = plt.gcf() #get current figure
#fig.set_size_inches(10,10)
#plt.savefig("tweets_word_cloud.png", format="png")

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.
  app.launch_new_instance()


In [ ]:
pd.set_option('display.max_colwidth', -1)
ftext_tweets[ftext_tweets['tweet_text'].str.contains("now")].sample(10)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,tweet_text
date,
2019-07-10 16:26:00,"@JohnSnow2019 I have cash and buying power on hand to handle it. Sharpened all my tools throughout the years where I think I would make off pretty well. I have portfolio margin, can trade futures, options, volatility. Been waiting for one since the last Great Recession."
2019-08-15 15:22:00,@WSJ They absolutely should...i know in this country we feel we can do and say whatever we want..but why should they be allowed into a country they wanted to hurt the economy on with their bds movement...words are words but when you vote to boycott the countries business you are not
2019-08-12 17:57:00,@Pricd05 @ChineseSnowden @DEAURANGZEB @godeepak217 @MeRadhiika Lets see... i feel khaleel is the one who gets kicked on the teeth everytime i mean see his economy ...but he manages to take wickets somehow
2019-08-14 16:21:00,"@realDonaldTrump Sorry to see the stock market falling. My 401K is suffering. But the good news is this reduces your only claim to fame, even though actually Obama is due the credit. This ensures you will lose in 2020 and likely go to prison. So thanks to you, I now smile when I see my 401K fall."
2019-06-27 17:09:00,"RT acorio ""To help you get a quick look into servicenow, here are three key stock market metrics and their definitions. https://t.co/itDT9gWnDb #ServiceNow"""
2019-08-15 15:26:00,@catherineVsnow @liberalincincy @RepDonBeyer Stock market we are not the world police?? But keep up those record high numbers.. Thanks Trump
2019-07-10 20:47:00,@Jonath8888Lewis @acnewsitics @realDonaldTrump What exactly had he done?\nDon't say the economy because he inherited that. Don't say unemployment because I know people in positions that told me they aren't extending it so people just can't collect. Also Trump doesn't control the stock market. The stock holders do.
2019-08-29 19:01:00,Is a global recession now the best chance of forcing a resolution to the escalating trade conflict between the United States and China? https://t.co/UbenuQaDr3 https://t.co/M9pDj0XJED
2019-08-20 16:52:00,"@Jim_Jordan No. Down turns happen. It is cyclical. You know that. Having a plan to combat a recession is important. ""Marketing"" how great the economy does not constitute a plan. I know he's your guy but try to remember when truth, fundamentals and integrity meant something. Wish for honesty."


In [ ]:
# import nltk; nltk.download('popular')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia=SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...


In [ ]:
del sia.lexicon['thank']
del sia.lexicon['thanks']

In [ ]:
stock_lex = pd.read_csv('stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
#filtering out multiple words expressions from the lexicon
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

In [ ]:
positive = []
with open('lm_positive.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        positive.append(row[0].strip())
        
negative = []
with open('lm_negative.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        entry = row[0].strip().split(" ")
        if len(entry) > 1:
            negative.extend(entry)
        else:
            negative.append(entry[0])

In [ ]:
final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [ ]:
final_lex

{'ABLE': 2.0,
 'ABUNDANCE': 2.0,
 'ABUNDANT': 2.0,
 'ACCLAIMED': 2.0,
 'ACCOMPLISH': 2.0,
 'ACCOMPLISHED': 2.0,
 'ACCOMPLISHES': 2.0,
 'ACCOMPLISHING': 2.0,
 'ACCOMPLISHMENT': 2.0,
 'ACCOMPLISHMENTS': 2.0,
 'ACHIEVE': 2.0,
 'ACHIEVED': 2.0,
 'ACHIEVEMENT': 2.0,
 'ACHIEVEMENTS': 2.0,
 'ACHIEVES': 2.0,
 'ACHIEVING': 2.0,
 'ADEQUATELY': 2.0,
 'ADVANCEMENT': 2.0,
 'ADVANCEMENTS': 2.0,
 'ADVANCES': 2.0,
 'ADVANCING': 2.0,
 'ADVANTAGE': 2.0,
 'ADVANTAGED': 2.0,
 'ADVANTAGEOUS': 2.0,
 'ADVANTAGEOUSLY': 2.0,
 'ADVANTAGES': 2.0,
 'ALLIANCE': 2.0,
 'ALLIANCES': 2.0,
 'ASSURE': 2.0,
 'ASSURED': 2.0,
 'ASSURES': 2.0,
 'ASSURING': 2.0,
 'ATTAIN': 2.0,
 'ATTAINED': 2.0,
 'ATTAINING': 2.0,
 'ATTAINMENT': 2.0,
 'ATTAINMENTS': 2.0,
 'ATTAINS': 2.0,
 'ATTRACTIVE': 2.0,
 'ATTRACTIVENESS': 2.0,
 'BEAUTIFUL': 2.0,
 'BEAUTIFULLY': 2.0,
 'BENEFICIAL': 2.0,
 'BENEFICIALLY': 2.0,
 'BENEFIT': 2.0,
 'BENEFITED': 2.0,
 'BENEFITING': 2.0,
 'BENEFITTED': 2.0,
 'BENEFITTING': 2.0,
 'BEST': 2.0,
 'BETTER': 2.0,
 'BOL

In [ ]:
sentiment=[]
for tweet in ftext_tweets['tweet_text']:
    sentiment.append(sia.polarity_scores(tweet)['compound'])
ftext_tweets['Sentiment Score']=sentiment

In [ ]:
pd.set_option('display.max_colwidth', -1)
ftext_tweets[ftext_tweets['tweet_text'].str.len()<150].sample(10)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,tweet_text,Sentiment Score
date,,
2019-08-12 15:46:00,@realDonaldTrump hows the Dow Jones doing now fuck tard?,-0.7878
2019-08-29 15:12:00,"@realDonaldTrump The stock market is not the Economy! Read a book, Donald.",0.2155
2019-07-16 21:04:00,Economic Report: Import prices post biggest drop in June in six months; more evidence of falling inflation https://t.co/TiSXgyySjB,-0.4381
2019-08-14 15:22:00,YIELD CURVE INVERTS: Recession indicator flashes red for first time since 2005 https://t.co/NSDrjaqp14,-0.4861
2019-06-28 16:34:00,#volkswagon news from @Autoblog- VW's Traton truck unit stalls on stock market debut https://t.co/MG3NkAOpNP,0.6820
2019-08-15 15:36:00,When this president tweets the stock market loses money. He panic's traders with his spastic mind set.,0.1921
2019-07-01 17:40:00,@PlanetPonzi Nice Head &amp; Shoulders formation. Wonder how long the Stock Buybacks can elevate the market.,0.7336
2019-08-21 18:51:00,#KingOfIsrael is what #Trump is blurting out now so we lose focus on the trade war that is bringing a recession.,-0.8031
2019-07-18 15:43:00,@realDonaldTrump full margin seeeeeelllllll dow jones perfect perfect,0.8627


In [ ]:
# Downsample the tweet sentiment scores
iss = ftext_tweets['Sentiment Score'].resample(freq,base=15.5,label='right').mean().to_frame()
iss = iss[iss['Sentiment Score'].notnull()]
iss['Z Scores']=(iss['Sentiment Score']-iss['Sentiment Score'].mean())/iss['Sentiment Score'].std()

In [ ]:
iss

,Sentiment Score,Z Scores
date,,
2019-06-24 20:30:00,0.063693,0.105390
2019-06-24 21:30:00,0.132368,0.803242
2019-06-25 15:30:00,0.172388,1.209915
2019-06-25 16:30:00,0.198150,1.471695
2019-06-25 17:30:00,0.132584,0.805442
...,...,...
2019-09-03 17:30:00,-0.055798,-1.108834
2019-09-03 18:30:00,-0.010625,-0.649796
2019-09-03 19:30:00,0.027969,-0.257626


In [ ]:
ftext_tweets

,tweet_text,Sentiment Score
date,,
2019-06-24 19:58:00,Nouriel Roubini outlines the 2020 recession risk.\nThis is my FT Alphachat podcast. Plus my views on Facebook's Libra coin \n https://t.co/QpRRmEetk4,-0.0400
2019-06-24 19:58:00,"@nyika_yedu The multi currency regime dealt with inflation, when the exchange rate is changing 3 times a day retailers are better not stocking than lose value on their stock. The Gvt is going to print whenever they are cornered &amp; rates will skyrocket",0.7282
2019-06-24 19:58:00,@jwoltmann @starboard_light People ARE paying for their loans and cannot get out from under them. It is a systemic problem for a generation that graduated at the time of the Great Recession. Unless you are a buying stocks in huge quantities YOU personally are not paying for anything. Read his plan,0.7321
2019-06-24 19:58:00,"The caution continues. ""Small business owners have been conservative about hiring since the recession. A hoped-for boom in small company hiring never happened because owners haven't wanted to take on the added risk and expense of more staffers.""\n\n https://t.co/jwebktIhA8",-0.0587
2019-06-24 19:59:00,Mapping The World's Financial Weak Spots - https://t.co/NVZd1jCmI1\n\n$SPY $SPX $ES_F $QQQ $NQ_F $DIA $DJIA $TLT $ZN_F $ZB_F,-0.4379
...,...,...
2019-09-03 20:47:00,"@PrimeXbt In general, the SP500 is clearly overrated https://t.co/YOzStZn4Gu",0.4051
2019-09-03 20:47:00,@AmandaAgatiPNC @NFIB Are you a recession denier? The bond market is screaming recession. I don't think it is different this time...,-0.8789
2019-09-03 20:47:00,"@pdtugs @CPC_HQ Actually Canada economy is good, American is headed to a recession thanks to Trump. Once again, good stewards of the economy, Liberals.",0.5850
